In [1]:
# %%writefile Ind_xxxxx.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat


In [ ]:
# Amihud定义了ILLIQ度量
# 一个粗略而实用的价格影响指标考虑了一段时间内绝对股票收益率与美元成交量的日平均比率。遵循Kyle的流动性概念，
# ILLIQ = 1/n * sum_t(|r_t|/v_t)

In [ ]:
# 均衡度量
# 基于均衡价格的测度试图捕捉向均衡价格的有序移动，主要是为了衡量弹性
# 市场效率系数（MEC）用于衡量价格变动的连续性：
# MEC = Var(R_t) / N*Var(r_t)

# Var（Rt）是对数长周期收益的方差，Var（Rt）是对数短周期收益的方差，N是每个长周期内的短周期数。

# 在更具弹性的市场中，这一比率往往接近但略低于1。低市场弹性意味着过度的短期波动，而价格四舍五入、利差和不准确的价格发现等因素将推动MEC远低于1。
# 做市商干预和价格决定不准确后对新信息的部分调整等因素将抑制短期波动，导致MEC升至1以上。

In [ ]:
# 市场调整度量
# 市场调整措施试图区分流动性引起的价格变动和一般市场状况或新信息到达等因素引起的价格变动。当新的信息可用时，即使是小的交易量也可能与大的价格变动相关联。

# 在股票市场上，经常根据资本资产定价模型（CAPM）区分系统性风险和非系统性风险，CAPM也提供了一种提取市场波动的方法。
# 系统性风险不能分散，因为它影响到所有证券。系统风险的程度称为股票的β，它是指股票的日收益率Ri与市场Rm的日收益率Ri的回归系数：
# Ri= α + β*R_m+ui

# 然后，使用回归残差ui将其方差与交易额的每日百分比变化联系起来

# ui^2 = γi + γ2*ΔVi+ ei
# 经市场调整的流动性利用资产回报率与市场回报率之间的回归残差（从而将其从系统风险中清除）来确定资产的内在流动性。
# γ2越小，交易量对资产价格波动性的影响越小，这意味着资产是流动的。系数越低，市场的宽度就越大。

# 没有一个单一的衡量标准明确地衡量紧密性、即时性、深度、广度和弹性。
# 然而，从从业者的角度来看，基于交易难度的证券聚类是可能的，这取决于许多因素，但主要取决于波动性、盘口价差、价格、排队规模、成交量等。



In [57]:
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

tail = True
if tail:
    file = 'all_tail_qfq'
else:
    file = 'all_train_qfq'
    
df_all = load_cache(file,cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(df_all)

data_ = df_all

# year1 = '2022'
# year2 = '2022'
# data_ = df_all.loc[year1+'-01-03':year2+'-12-31']
# codes = data_.index.get_level_values(1).unique()
# dates = data_.index.get_level_values(0).unique()


# ret = smpl.get_forward_return(data_,'close')
# ret.name = 'ret_forward'
# ret[np.abs(ret)>0.15]=np.nan

In [122]:
def get_future_period_return(stocks_df,column,forward_window,method='sum'):
    '''计算未来一定区间的（总）回报率
    :param stocks_df: {pd.DataFrame 或 stock_struct}
    :param column: {string}
    :param forward_window: {int} --前望窗口长度
    :param method: {str in ['mean', 'sum', 'std', 'median', 'skew', 'kurt', 'min', 'max']} --计算方法，
    :return: {pd.Series}
    '''
    assert method in ['mean', 'sum', 'std', 'median', 'skew', 'kurt', 'min', 'max'], "method MUST Be one of ['mean', 'sum', 'std', 'median', 'skew', 'kurt', 'min', 'max']}"
    def calc(x):
        ret = (x/x.shift(1)-1).shift(-1)
        if method in ['sum']:
            ret = np.log(ret+1)
            
        ret = ret[::-1].rolling(forward_window).agg(method)[::-1]
        
        if method in ['sum']:
            ret = np.exp(ret)-1
        
        return ret
        
    ret = stocks_df[column].groupby(level=1, group_keys=False).apply(calc)
    ret.name = 'ret_forward'
    return ret

In [84]:
test=data_.loc[(slice('2021-01-04','2021-02-04'),'000001'),]

(test['close']/test['close'].shift(1)-1).shift(-1)

date        code  
2021-01-04  000001   -0.023118
2021-01-05  000001    0.076500
2021-01-06  000001    0.017382
2021-01-07  000001   -0.002513
2021-01-08  000001    0.026700
2021-01-11  000001    0.030422
2021-01-12  000001   -0.014286
2021-01-13  000001   -0.025604
2021-01-14  000001    0.041150
2021-01-15  000001    0.080952
2021-01-18  000001   -0.015859
2021-01-19  000001    0.005819
2021-01-20  000001   -0.010681
2021-01-21  000001   -0.008997
2021-01-22  000001    0.020881
2021-01-25  000001   -0.005336
2021-01-26  000001    0.031739
2021-01-27  000001   -0.011698
2021-01-28  000001    0.012275
2021-01-29  000001    0.063231
2021-02-01  000001   -0.051731
2021-02-02  000001    0.071736
2021-02-03  000001   -0.014028
2021-02-04  000001         NaN
Name: close, dtype: float32

In [119]:
get_future_period_return(test,'close',4,method='sum')

date        code  
2021-01-04  000001    0.067204
2021-01-05  000001    0.121629
2021-01-06  000001    0.073620
2021-01-07  000001    0.040201
2021-01-08  000001    0.016121
2021-01-11  000001    0.030422
2021-01-12  000001    0.080952
2021-01-13  000001    0.079227
2021-01-14  000001    0.114031
2021-01-15  000001    0.058571
2021-01-18  000001   -0.029515
2021-01-19  000001    0.006715
2021-01-20  000001   -0.004450
2021-01-21  000001    0.038237
2021-01-22  000001    0.035406
2021-01-25  000001    0.026678
2021-01-26  000001    0.097452
2021-01-27  000001    0.008665
2021-01-28  000001    0.093819
2021-01-29  000001    0.065396
2021-02-01  000001         NaN
2021-02-02  000001         NaN
2021-02-03  000001         NaN
2021-02-04  000001         NaN
Name: ret_forward, dtype: float64

In [121]:
get_future_period_return(test,'close',4,method='sum')

date        code  
2021-01-04  000001    0.068251
2021-01-05  000001    0.118070
2021-01-06  000001    0.071992
2021-01-07  000001    0.040324
2021-01-08  000001    0.017233
2021-01-11  000001    0.031683
2021-01-12  000001    0.082213
2021-01-13  000001    0.080640
2021-01-14  000001    0.112063
2021-01-15  000001    0.060232
2021-01-18  000001   -0.029718
2021-01-19  000001    0.007022
2021-01-20  000001   -0.004133
2021-01-21  000001    0.038287
2021-01-22  000001    0.035586
2021-01-25  000001    0.026980
2021-01-26  000001    0.095547
2021-01-27  000001    0.012076
2021-01-28  000001    0.095510
2021-01-29  000001    0.069207
2021-02-01  000001         NaN
2021-02-02  000001         NaN
2021-02-03  000001         NaN
2021-02-04  000001         NaN
Name: ret_forward, dtype: float64

In [114]:
a = -0.05
b = 0.07
print(a+b)
print(np.exp(np.log(a+1) + np.log(b+1))-1)

0.020000000000000004
0.01649999999999996


In [102]:
np.exp(np.log(a))

2499.9999999999995